In [ ]:
import numpy as np


class Deck:

    ranks = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    suits = ('D', 'H', 'C', 'S')
    cards = tuple(str(v) + s for v in ranks for s in ('D', 'H', 'C', 'S'))
    colors = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        self._stock = self.shuffle(self.cards * num_packs)
        self._drawn = []

    def draw(self, n):
        cards = [self._stock.pop(0) for _ in range(n)]
        self._drawn += cards
        return cards

    def reset(self):
        self._stock = self.shuffle(self._stock + self._drawn)
        self._drawn = []

    def __getitem__(self, index):
        return self.stock[index]

    def __eq__(self, other_deck):
        return len(self) == len(other_deck) and all([c1 == c2 for c1, c2
                                                 in zip(self, other_deck)])

    def __contains__(self, card):
        return card in self.stock

    def __repr__(self):
        return f'Deck({len(self)})'

    def __len__(self):
        return len(self.stock)

    @property
    def stock(self):
        return self._stock

    @property
    def drawn(self):
        return self._drawn

    @staticmethod
    def shuffle(cards):
      return list(np.random.permutation(cards))

In [ ]:
class CardGame:

    def __init__(self, num_players=2, num_packs=5):
        self._num_players = num_players
        self._deck = Deck(num_packs=num_packs)
        self._player_hands = [[] for _ in range(num_players)]

    def draw(self, num_cards=1, burn=False):
        cards = self._deck.draw(num_cards)
        if burn:
            self._deck.draw()
        return cards

    def deal(self, num_cards, burn=False):
        return list(zip(*[self.draw(self.num_players, burn=burn) for i in range(num_cards)]))

    def update_players_hands(self, num_cards, burn=False):
        cards = self.deal(num_cards=num_cards, burn=burn)
        for player_hand, player_new_cards in zip(self.player_hands, cards):
            player_hand.extend(list(player_new_cards))

    def get_hand(self, player_id):
        return self.player_hands[player_id]

    @property
    def num_players(self):
        return self._num_players

    @property
    def deck(self):
        return self._deck

    @property
    def player_hands(self):
        return self._player_hands

    def reset(self):
      num_packs = len(self.deck) // len(self.deck.cards)
      self.__init__(num_players=self.num_players, num_packs=num_packs)

In [ ]:
from abc import ABC, abstractmethod


class BlackJackPlayer(ABC):

    ALLOWED_MOVES = ('HIT', 'STAND')

    @abstractmethod
    def strategy(self, hand_value):
        pass

    def is_valid_move(play_func):
        def wrapper(*args, **kwargs):
            move = play_func(*args, **kwargs)
            if move in BlackJackPlayer.ALLOWED_MOVES:
              return move
            raise ValueError(f'Invalid move. Only values {BlackJackPlayer.ALLOWED_MOVES} are allowed')
        return wrapper

    @is_valid_move
    def play(self, hand_value):
        return self.strategy(hand_value)

    def __repr__(self):
        return self.__class__.__name__

class GreedyBlackJackPlayer(BlackJackPlayer):

    def strategy(self, hand_value):
        if hand_value > 19:
            return 'STAND'
        else:
            return 'HIT'

class InvalidMovePlayer(BlackJackPlayer):
    def strategy(self, hand_value):
        if hand_value > 19:
            return 'STAND'
        elif hand_value == 15:
            return 'INVALID'
        else:
            return 'HIT'


class CautiousBlackJackPlayer(BlackJackPlayer):

    def strategy(self, hand_value):
        if hand_value > 15:
            return 'STAND'
        else:
            return 'HIT'


class BlackJackDealer(BlackJackPlayer, ABC):

    ALLOWED_MOVES = ('HIT', 'STAND')

    @abstractmethod
    def strategy(self, hand_value, player_values):
        pass

    @BlackJackPlayer.is_valid_move
    def play(self, hand_value, player_values):
        return self.strategy(hand_value, player_values)


class BeatOneBlackJackDealer(BlackJackDealer):

    def strategy(self, hand_value, player_values):
        if any([hand_value >= player_value for player_value in player_values]):
            return 'STAND'
        return 'HIT'


class BeatAllBlackJackDealer(BlackJackDealer):

    def strategy(self, hand_value, player_values):
        if all([hand_value >= player_value for player_value in player_values]):
            return 'STAND'
        return 'HIT'

def player_factory(player_type):
    players = {'greedy': GreedyBlackJackPlayer,
               'cautious': CautiousBlackJackPlayer}
    return players[player_type]()

def dealer_factory(dealer_type):
    dealers = {'beat_one': BeatOneBlackJackDealer,
               'beat_all': BeatAllBlackJackDealer}
    return dealers[dealer_type]()

In [ ]:
from types import resolve_bases
class Blackjack(CardGame):

    card_values = {str(c): c for c in range(2, 11)}
    card_values.update({c: 10 for c in ('J', 'Q', 'K')})
    card_values.update({'A': 11})  # I simplified the game a bit

    def __init__(self, player_types, dealer_type, num_packs=5, verbose=0):
        super().__init__(num_players=len(player_types)+1, num_packs=num_packs)
        self._players = [player_factory(player_type) for player_type in player_types]
        self._dealer = dealer_factory(dealer_type)
        self.verbose = verbose

    def ask_player(self, player_id):
        player_hand = self.player_hands[player_id]
        player_hand_value = self.calculate_value(player_hand)
        return self.players[player_id].play(player_hand_value) if player_hand_value else 'STAND'

    def ask_dealer(self, player_hands):
        dealer_hand = self.player_hands[-1]
        dealer_hand_value = self.calculate_value(dealer_hand)
        return self.dealer.play(dealer_hand_value, player_hands) if dealer_hand_value else 'STAND'

    def play_player_round(self, player_id):
        while self.ask_player(player_id) == 'HIT':
            new_card = self.draw()[0]
            self.player_hands[player_id].append(new_card)
            if self.verbose:
                print(f'  Player {player_id+1} decides to "HIT", got {new_card} for a '\
                f'total value of: {self.calculate_value(self.player_hands[player_id])}')

        current_value = self.calculate_value(self.player_hands[player_id])
        if self.verbose and current_value:
            print(f'  Player {player_id+1} decides to "STAND"')
        return current_value

    def play_dealer_round(self, player_hands):
        if self.verbose:
            print(f'  Dealer reveals his cards and has {self.player_hands[-1]} for '\
                  f'a total value of: {self.calculate_value(self.player_hands[-1])}')

        while self.ask_dealer(player_hands) == 'HIT':
            new_card = self.draw()[0]
            self.player_hands[-1].append(new_card)
            if self.verbose:
                print(f'  Dealer decides to "HIT", got {new_card} for a '\
                f'total value of: {self.calculate_value(self.player_hands[-1])}')

        current_value = self.calculate_value(self.player_hands[-1])
        if self.verbose and current_value:
            print(f'  Dealer decides to "STAND"')
        return self.calculate_value(self.player_hands[-1])

    def play_full_round(self):

        self._player_hands = [[] for _ in range(self.num_players)]
        self.update_players_hands(2)

        if self.verbose:
            print('----------')
            print('Dealing...')
            print('----------')

            for i, cards in enumerate(self.player_hands[:-1]):
                print(f'  Player {i+1} got {cards} for a total value of: {self.calculate_value(cards)}')

            print('------------')
            print('Player turns')
            print('------------')

        player_hand_values = [self.play_player_round(player_id)
                              for player_id in range(self.num_players-1)]

        if self.verbose:
            print('-----------')
            print('Dealer turn')
            print('-----------')

        dealer_hand_value = self.play_dealer_round(player_hand_values)

        results = [self.resolve_hand(value, dealer_hand_value) for value in player_hand_values]

        if self.verbose:
            print('------')
            print('Result')
            print('------')

            outcomes = {False: 'lost', True: 'won'}

            for i, outcome in enumerate(results):
                print(f' Player {i+1} {outcomes[outcome]}')

        return results

    def resolve_hand(self, value, dealers_value=21):
        if dealers_value:
            if value > dealers_value:
                return True
            else:
                return False
        else:
            if value > 21:
                return False
            if value == 21:
                return True
            return value

    def new_round(self, num_players=None):
      self.reset()

    @property
    def players(self):
        return self._players

    @property
    def dealer(self):
        return self._dealer

    @staticmethod
    def calculate_value(cards):
        raw_value = sum([Blackjack.card_values[card[:-1]] for card in cards])
        return raw_value if raw_value <=22 else 0

In [ ]:
bj = Blackjack(['greedy', 'greedy', 'cautious', 'cautious'], 'beat_one', verbose=1)

In [ ]:
bj.players, bj.dealer

([GreedyBlackJackPlayer,
  GreedyBlackJackPlayer,
  CautiousBlackJackPlayer,
  CautiousBlackJackPlayer],
 BeatOneBlackJackDealer)

In [ ]:
bj.player_hands

[[], [], [], [], []]

In [ ]:
bj.play_full_round()

----------
Dealing...
----------
  Player 1 got ['4H', '4D'] for a total value of: 8
  Player 2 got ['9H', '9C'] for a total value of: 18
  Player 3 got ['10D', '4D'] for a total value of: 14
  Player 4 got ['7C', '4C'] for a total value of: 11
------------
Player turns
------------
  Player 1 decides to "HIT", got 5C for a total value of: 13
  Player 1 decides to "HIT", got 6S for a total value of: 19
  Player 1 decides to "HIT", got 4D for a total value of: 0
  Player 2 decides to "HIT", got 2S for a total value of: 20
  Player 2 decides to "STAND"
  Player 3 decides to "HIT", got 7C for a total value of: 21
  Player 3 decides to "STAND"
  Player 4 decides to "HIT", got 2D for a total value of: 13
  Player 4 decides to "HIT", got 7S for a total value of: 20
  Player 4 decides to "STAND"
-----------
Dealer turn
-----------
  Dealer reveals his cards and has ['AH', '10S'] for a total value of: 21
  Dealer decides to "STAND"
------
Result
------
 Player 1 lost
 Player 2 lost
 Player 3 l

[False, False, False, False]

In [ ]:
len(bj.deck.drawn)

83